In [11]:
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(data.table))
suppressPackageStartupMessages(library(parallel))
suppressPackageStartupMessages(library(reshape2))
suppressPackageStartupMessages(library(parallel))
suppressPackageStartupMessages(library(foreach))
suppressPackageStartupMessages(library(tidyr))
suppressPackageStartupMessages(library(tibble))
suppressPackageStartupMessages(library(ggplot2))

In [8]:
# Read the meta data
sanctumdbcon <- RMariaDB::dbConnect(
    drv = RMariaDB::MariaDB(),
    dbname = "Devdb",
    user = "admin",
    password = "JohnLennon1980",
    host = "sanctum-instance-1.cnklj0yzvfnv.us-east-1.rds.amazonaws.com",
    port = 3306
  )
clin_data <- dbReadTable(sanctumdbcon, name = clin_table)
dbDisconnect(sanctumdbcon)

ERROR: Error in dbReadTable(sanctumdbcon, name = clin_table): object 'clin_table' not found


In [ ]:
clin_data <- dcast(subset(clin_data, !is.na(key)), 
                 formula = model_id ~ key,
                 fun.aggregate = max) %>%
subset(., Model %in% colnames(counts)) # keep metadata for relavant samples

colnames(clin_data) <- gsub('..via.TruSight.clinical.', '', colnames(clin_data))
clin_data[] <- lapply(clin_data, toupper)
clin_data[clin_data=="PENDING" | clin_data=="NOT AVAILABLE" | clin_data=="OTHER" ] <- NA
clin_data[] <- lapply(clin_data, function(x) gsub("DIFFERENTIATED", "DIFF.", x))
clin_data[] <- lapply(clin_data, function(x) gsub("SQUAMOUS CELL", "SC", x))
# remove (largely) empty columns 
clin_data <- clin_data[apply(clin_data, 2, function(x) sum(is.na(x)) < nrow(clin_data)/2)]
# identify numeric columns
clin_data <- retype(clin_data)

### Proteomics data

In [6]:
prot  <- read.csv("./data/Prot_Data_F20FTSUSAT1137_47AML_2021-03-29.csv")
head(prot, 2)

X,ProteinGroupId,ProteinGroup,Protein,SingleProtein,Sample,Intensity,Counts,ProteinGroupName,hgnc_symbol
1,A0A075B759,sp|A0A075B759|PAL4E_HUMAN;sp|F5H284|PAL4D_HUMAN,sp|A0A075B759|PAL4E_HUMAN,sp|A0A075B759|PAL4E_HUMAN,CTG-2226,17.225,153194,PAL4E_HUMAN,PPIAL4E
2,A0A075B759,sp|A0A075B759|PAL4E_HUMAN;sp|F5H284|PAL4D_HUMAN,sp|A0A075B759|PAL4E_HUMAN,sp|A0A075B759|PAL4E_HUMAN,CTG-2227,18.070,275177,PAL4E_HUMAN,PPIAL4E
